# Autómata Celular que modela el comportamiento de un virus en un cuerpo humano.

*Grupo 4*

- Este proyecto está pensado para ser ejecutado con 4 archivos, celula_sanguinea.py, estado_celular.py. grilla_torrente_sanguineo.py y simulador.py, fuera del entorno del notebook. Esto es debido a que nuestra interfaz gráfica utiliza Tkinter, y esta clase de UI no se puede realizar en Google Collab.
- Debido a ello, cada celda por separado lleva el código de su respectivo archivo, aún si da error al cambiar al entorno del notebook.

### Clase Estado Celular

In [2]:
# @title
from enum import Enum
from dataclasses import dataclass

@dataclass
class CellState:
    codigo: int
    color: tuple
    nombre: str

class EstadoCelular(Enum):

    # Células sanas - INCLUÍ ESPECIFICACIÓN DE LOS GLÓBULOS BLANCOS
    Plasma = CellState(0, (255, 240, 240), "Plasma")
    GlobuloRojo = CellState(1, (220, 20, 60), "Glóbulo Rojo (Eritrocito)")
    Neutrofilo = CellState(2, (200, 200, 255), "Neutrófilo")
    Linfocito = CellState(3, (100, 149, 237), "Linfocito")
    Macrofago = CellState(4, (70, 130, 180), "Macrófago")
    Plaqueta = CellState(5, (255, 182, 193), "Plaqueta")

    # Sistema inmune activado - Células Peleando
    NeutrofiloActivado = CellState(6, (0, 191, 255), "Neutrófilo Activado")
    LinfocitoActivado = CellState(7, (30, 144, 255), "Linfocito Activado")
    MacrofagoActivado = CellState(8, (0, 100, 200), "Macrófago Activado")

    # Proceso de infección
    ParticulaVirus = CellState(9, (255, 255, 0), "Partícula Viral")
    CelulaRojaInfectada = CellState(10, (255, 140, 0), "Glóbulo Rojo Infectado")
    CelulaBlancaInfectada = CellState(11, (255, 165, 0), "Glóbulo Blanco Infectado")
    ExplosionViral = CellState(12, (255, 69, 0), "Explosión Viral")

    # Transformación zombie
    GlobuloRojoZombie = CellState(13, (139, 0, 0), "Glóbulo Rojo Zombificado")
    GlobuloBlancoZombie = CellState(14, (128, 0, 0), "Glóbulo Blanco Zombificado")
    TejidoNecrotico = CellState(15, (80, 0, 0), "Tejido Necrótico")

    # Coagulación y barreras - Hacer que detecte obstáculos, para marcar el flujo celular
    CoaguloSangre = CellState(16, (100, 50, 50), "Coágulo de Sangre")
    ParedVaso = CellState(17, (0, 0, 0), "Pared de Vaso Sanguíneo")

    def obtenerCodigo(self):
        return self.value.codigo

    def obtenerColor(self):
        return self.value.color

    def obtenerNombre(self):
        return self.value.nombre

    def esCelulaInmune(self):
        return self in {
            EstadoCelular.Neutrofilo,
            EstadoCelular.Linfocito,
            EstadoCelular.Macrofago,
            EstadoCelular.NeutrofiloActivado,
            EstadoCelular.LinfocitoActivado,
            EstadoCelular.MacrofagoActivado
        }

    def estaInfectada(self):
        return self in {
            EstadoCelular.CelulaRojaInfectada,
            EstadoCelular.CelulaBlancaInfectada,
            EstadoCelular.ExplosionViral
        }

    def esZombie(self):
        return self in {
            EstadoCelular.GlobuloRojoZombie,
            EstadoCelular.GlobuloBlancoZombie,
            EstadoCelular.TejidoNecrotico
        }

    def esVirus(self):
        return self in {
            EstadoCelular.ParticulaVirus,
            EstadoCelular.ExplosionViral
        }

    def estaSana(self):
        return self in {
            EstadoCelular.GlobuloRojo,
            EstadoCelular.Neutrofilo,
            EstadoCelular.Linfocito,
            EstadoCelular.Macrofago,
            EstadoCelular.Plaqueta
        }




### Clase Célula Sanguínea

In [3]:
# @title
import random
from dataclasses import dataclass
# Para correr como archivo .py, descomentar la siguiente línea. Para correr como Collab, comentar la siguiente línea
#from estado_celular import EstadoCelular

class CelulaSanguinea:
    def __init__(self, estadoInicial:EstadoCelular):
        self.estadoActual = estadoInicial
        self.estadoSiguiente = estadoInicial
        self.edadEstado = 0 #MÁXIMO DE 100 PARA TODOS LOS SIGUIENTES
        self.cargaViral = 0
        self.nivelAnticuerpos = 20
        self.memoriaInmune = 0
        self.nivelEnergia = 100 #Energía para procesos de la célula (tipo ATP)

    def calcularSiguienteEstado(self, vecinos: list) -> EstadoCelular:
        analisis = self.analizarVecindario(vecinos)
        self.actualizarParametrosLocales(analisis)

        #Posibles Estados - Variaciones
        if self.estadoActual == EstadoCelular.Plasma:
            return self.aplicarReglasPlasma(analisis)
        elif self.estadoActual == EstadoCelular.GlobuloRojo:
            return self.aplicarReglasGlobuloRojo(analisis)
        elif self.estadoActual in (EstadoCelular.Neutrofilo, EstadoCelular.NeutrofiloActivado):
            return self.aplicarReglasNeutrofilo(analisis)
        elif self.estadoActual in (EstadoCelular.Linfocito, EstadoCelular.LinfocitoActivado):
            return self.aplicarReglasLinfocito(analisis)
        elif self.estadoActual in (EstadoCelular.Macrofago, EstadoCelular.MacrofagoActivado):
            return self.aplicarReglasMacrofago(analisis)
        elif self.estadoActual == EstadoCelular.Plaqueta:
            return self.aplicarReglasPlaqueta(analisis)
        elif self.estadoActual == EstadoCelular.ParticulaVirus:
            return self.aplicarReglasParticulaVirus(analisis)
        elif self.estadoActual in (EstadoCelular.CelulaRojaInfectada, EstadoCelular.CelulaBlancaInfectada):
            return self.aplicarReglasCelulaInfectada(analisis)
        elif self.estadoActual == EstadoCelular.ExplosionViral:
            return self.aplicarReglasExplosionViral(analisis)
        elif self.estadoActual in (EstadoCelular.GlobuloRojoZombie, EstadoCelular.GlobuloBlancoZombie):
            return self.aplicarReglasZombie(analisis)
        elif self.estadoActual == EstadoCelular.TejidoNecrotico:
            return self.aplicarReglasNecrotico(analisis)
        elif self.estadoActual == EstadoCelular.CoaguloSangre:
            return self.aplicarReglasCoagulo(analisis)
        elif self.estadoActual == EstadoCelular.ParedVaso:
            return EstadoCelular.ParedVaso
        else:
            return self.estadoActual

    def analizarVecindario(self, vecinos:list):
        analisis = self.analisisVecindario()

        for vecino in vecinos:
            if vecino is None:
                continue

            estado = vecino.obtenerEstadoActual()

            if estado.esVirus():
                analisis.cantidadVirus += 1
            if estado.estaInfectada():
                analisis.cantidadInfectadas += 1
            if estado.esZombie():
                analisis.cantidadZombies += 1
            if estado.esCelulaInmune():
                analisis.cantidadInmunes += 1
                analisis.totalAnticuerpos += vecino.nivelAnticuerpos
            if estado == EstadoCelular.GlobuloRojo:
                analisis.cantidadGlobulosRojos += 1
            if estado == EstadoCelular.Plaqueta:
                analisis.cantidadPlaquetas += 1
            if estado == EstadoCelular.CoaguloSangre:
                analisis.cantidadCoagulos += 1
            if estado == EstadoCelular.Plasma:
                analisis.cantidadPlasma += 1

            # Acumular carga viral del vecindario
            analisis.cargaViralVecindario += vecino.cargaViral

        return analisis

    def actualizarParametrosLocales(self, analisis: 'analisisVecindario'):

        #Incremento y actualización de la carga viral
        self.cargaViral += (analisis.cantidadVirus * 12 + analisis.cantidadInfectadas * 6 + analisis.cantidadZombies *10)
        self.cargaViral -= ((analisis.cantidadInmunes *  18 + analisis.totalAnticuerpos) // 10 )
        self.cargaViral = max(0, min(100, self.cargaViral))

        #Actualizacion del nivel de anticuerpos
        if analisis.cantidadInmunes > 0:
            self.nivelAnticuerpos = self.nivelAnticuerpos + (analisis.totalAnticuerpos / 20)
            self.nivelAnticuerpos = min(100, self.nivelAnticuerpos)

        #Energía que se gasta con la edad y la infección
        self.nivelEnergia -= (self.cargaViral / 50)
        self.nivelEnergia = max(0, self.nivelEnergia)

    def aplicarReglasPlasma(self, a : 'analisisVecindario') -> EstadoCelular:

        #Regla: El plasma puede contaminarse con el virus.
        if a.cantidadVirus >= 2 or self.cargaViral > 30:
            return EstadoCelular.ParticulaVirus

        #Regla: Coagulación defensiva ante infección masiva.
        if a.cantidadPlaquetas >= 4 and (a.cantidadInfectadas + a.cantidadZombies) > 2:
            return EstadoCelular.CoaguloSangre

        return EstadoCelular.Plasma

    def aplicarReglasGlobuloRojo(self, a: 'analisisVecindario') -> EstadoCelular:

        #Parámetro : Umbral de infección basado en la exposición a la carga viral.
        riesgoInfeccion =  (a.cantidadVirus * 0.25) + (a.cantidadInfectadas * 0.15) + (self.cargaViral / 200.0)

        #Regla: Si se genera un número random menor al riesgo de infección, el globulo rojo se cambia de estado a infectada.
        if random.random() < riesgoInfeccion:
            return EstadoCelular.CelulaRojaInfectada

        #Regla: Zombificación directa su se encuentra rodeada completamente de células Zombies
        if a.cantidadZombies >= 6:
            return EstadoCelular.GlobuloRojoZombie

        #Regla: Protección por anticuerpos del vecindario.
        if self.nivelAnticuerpos > 60 and a.cantidadInmunes >= 3:
            self.cargaViral = max(0, self.cargaViral - 20)

        return EstadoCelular.GlobuloRojo

    def aplicarReglasNeutrofilo(self, a:'analisisVecindario') -> EstadoCelular:
        estaActivado = ( self.estadoActual == EstadoCelular.NeutrofiloActivado)

        #Regla: Actúan como respuesta rápida ante la amenaza del virus y células infectadas.
        if not estaActivado and (a.cantidadVirus + a.cantidadInfectadas >= 2):
            self.nivelEnergia = 100
            return EstadoCelular.NeutrofiloActivado

        if estaActivado:
            #Regla:  Defensa mediante proceso de fagocitosis - Destruye virus con probabilidad.
            if a.cantidadVirus > 0 and self.nivelEnergia > 30:
                probabilidadFagocitosis = 0.65 - (self.cargaViral/200.0)
                if random.random() < probabilidadFagocitosis:
                    self.cargaViral = 0
                    self.nivelEnergia -= 40
                    #Muerte de la celula después de la fagocitosis
                    if self.nivelEnergia < 20 or random.random() < 0.3:
                        return EstadoCelular.Plasma

            #Regla: Resistencia moderada a la infección.
            if a.cantidadZombies >= 4 or self.cargaViral > 75:
                probabilidadInfeccion = 0.35 + (self.cargaViral/150.0)
                if random.random() < probabilidadInfeccion:
                    return EstadoCelular.CelulaBlancaInfectada

            #Regla: Se desactiva como sistema de defensa si ya no hay amenaza.
            if a.cantidadVirus == 0 and a.cantidadInfectadas == 0 and self.edadEstado > 5:
                return EstadoCelular.Neutrofilo

        if not estaActivado and a.cantidadVirus >= 3 and self.cargaViral > 50:
            if random.random() < 0.25:
                return EstadoCelular.CelulaBlancaInfectada

        return self.estadoActual

    def aplicarReglasLinfocito(self, a: 'analisisVecindario') -> EstadoCelular:
        estaActivado = (self.estadoActual == EstadoCelular.LinfocitoActivado)

        # Regla: Activación más rápida con memoria alta (inmunidad entrenada)
        umbralActivacion = 2
        if self.memoriaInmune > 60:
            umbralActivacion = 1  # Se activa más rápido si "recuerda" el patógeno.

        # Regla: Respuesta más lenta del sistema pero duradera - Activación.
        if not estaActivado and (a.cantidadInfectadas + a.cantidadZombies >= umbralActivacion or self.cargaViral > 40):
            self.nivelAnticuerpos += 25
            self.memoriaInmune += 15
            return EstadoCelular.LinfocitoActivado

        if estaActivado:
            # Producción de anticuerpos, se da una inmunidad adaptativa.
            #Regla: Producción acelerada con alta memoria.
            incrementoAnticuerpos = 5
            if self.memoriaInmune > 70:
                incrementoAnticuerpos = 8

            self.nivelAnticuerpos = min(100, self.nivelAnticuerpos + incrementoAnticuerpos)
            self.memoriaInmune = min(100, self.memoriaInmune + 3)

            # Regla: Curación de células infectadas mediante anticuerpos
            if a.cantidadInfectadas > 0 and self.nivelAnticuerpos > 70:
                #Proceso: Curación más efectiva con memoria alta.
                bonusMemoria = self.memoriaInmune / 200.0  # OJO: 0 a 0.5 de bonus.
                probabilidadCuracion = (0.35 + bonusMemoria) * (self.nivelAnticuerpos / 100.0)

                #Regla: Reduce más carga viral si tiene memoria
                if random.random() < probabilidadCuracion:
                    reduccionViral = 30 + (self.memoriaInmune // 5)  # 30-50 según memoria
                    self.cargaViral = max(0, self.cargaViral - reduccionViral)

            # Regla: Alta resistencia pero sin inmunidad. - Más resistente con memoria alta
            resistenciaBase = 0.2
            if self.memoriaInmune > 50:
                resistenciaBase = 0.1  # Menos probabilidad de infectarse

            if a.cantidadZombies >= 5 and self.cargaViral > 85:
                if random.random() < resistenciaBase:
                    return EstadoCelular.CelulaBlancaInfectada

            # Regla: Si la amenaza disminuye, se desactivan progresivamente. - Con memoria alta, permanece activado más tiempo (vigilancia)
            tiempoDesactivacion = 8
            if self.memoriaInmune > 80:
                tiempoDesactivacion = 15

            if a.cantidadVirus == 0 and a.cantidadInfectadas == 0 and a.cantidadZombies == 0:
                if self.edadEstado > tiempoDesactivacion:
                    return EstadoCelular.Linfocito

        # Regla: Linfocitos inactivos son más resistentes que los neutrófilos (Durabilidad)- Memoria residual protege incluso inactivos
        resistenciaInactivo = 0.15
        if self.memoriaInmune > 40:
            resistenciaInactivo = 0.08  # Mucho más resistente con memoria

        if not estaActivado and a.cantidadVirus >= 4 and self.cargaViral > 65:
            if random.random() < resistenciaInactivo:
                return EstadoCelular.CelulaBlancaInfectada

        return self.estadoActual

    def aplicarReglasMacrofago(self, a:'analisisVecindario') -> EstadoCelular:

        estaActivado = ( self.estadoActual == EstadoCelular.MacrofagoActivado)

        #Regla: Alto requerimiento de virus y celulas infectadas para su activación - Un macrófago es grande, se mueve lento, reacciona más tarde.
        if not estaActivado and (a.cantidadVirus + a.cantidadInfectadas >=  3):
            self.nivelEnergia = 100
            return EstadoCelular.MacrofagoActivado

        if estaActivado:

            #Regla: Se hace fagocitosis de las células infectadas y  virus.
            if a.cantidadInfectadas > 0 and self.nivelEnergia > 40:
                probabilidadFagocitosis = 0.55
                if random.random() < probabilidadFagocitosis:
                    self.cargaViral = max(0, self.cargaViral - 25)
                    self.nivelEnergia -= 30

                    #Regla: Muerte del macrofago después de ejecutar ese proceso muchas veces.
                    if self.nivelEnergia < 25 or self.edadEstado > 15:
                        return EstadoCelular.Plasma

            #Regla: Entrenamiento de la memoria inmune.
            if a.cantidadInmunes > 0 and self.edadEstado % 3 == 0:
                self.memoriaInmune += 10

            #Regla: Alta resistencia.
            if a.cantidadZombies >= 6 and self.cargaViral > 90:
                probabilidadInfeccion = 0.15 + (self.cargaViral/300.0)
                if random.random() < probabilidadInfeccion:
                    return EstadoCelular.CelulaBlancaInfectada

            #Regla: Desactivación Lenta de los macrófagos.
            if (a.cantidadVirus + a.cantidadInfectadas == 0) and self.edadEstado > 10:
                return EstadoCelular.Macrofago

        return self.estadoActual

    def aplicarReglasPlaqueta(self, a : 'analisisVecindario') -> EstadoCelular:

        #Regla: Las plaquetas forman coágulos para contener la infección.
        if a.cantidadPlaquetas >= 3 and (a.cantidadInfectadas + a.cantidadZombies) > 1:
            probabilidadCoagulo = 0.3 + (a.cantidadZombies * 0.1)
            if random.random() < probabilidadCoagulo:
                return EstadoCelular.CoaguloSangre

        #Regla: Conversión a partículas de virus.
        if a.cantidadVirus >= 2 and self.cargaViral > 40:
            return EstadoCelular.ParticulaVirus

        return EstadoCelular.Plaqueta

    def aplicarReglasParticulaVirus(self, a: 'analisisVecindario') -> EstadoCelular:

        #Regla: Preferencia a la infección de glóbulos rojos.
        if a.cantidadGlobulosRojos > 0  and random.random() < 0.75:
            return EstadoCelular.CelulaRojaInfectada

        #Regla: Infecta células del sistema inmune si hay pocas rojas.
        if a.cantidadGlobulosRojos == 0 and a.cantidadInmunes > 0 and random.random() < 0.4:
            return EstadoCelular.CelulaBlancaInfectada

        #Regla: Elimina células del sistema inmune.
        if a.cantidadInmunes >= 2 or self.nivelAnticuerpos > 60:
            probabilidadElimminacion = 0.6 + (self.nivelAnticuerpos / 200)
            if random.random() < probabilidadElimminacion:
                return EstadoCelular.Plasma

        #Regla: Generación de explosión viral.
        if self.edadEstado > 4 and a.cantidadVirus < 3:
            if random.random() < 0.25:
                return EstadoCelular.ExplosionViral

        return EstadoCelular.ParticulaVirus

    def aplicarReglasCelulaInfectada(self, a:'analisisVecindario') -> EstadoCelular:

        esRojaInfectada = (self.estadoActual == EstadoCelular.CelulaRojaInfectada)

        #Regla: Según el tipo, se da el periodo de infección.
        periodoIncubacion = 3 if esRojaInfectada else 4

        #Regla: Generación de una replicación viral completa.
        if self.edadEstado >= periodoIncubacion:
            return EstadoCelular.ExplosionViral

        #Regla: Rescate del sujeto por parte ddel sistena inmune.
        probabilidadRescate = 0.25 -(self.edadEstado * 0.05)

        if esRojaInfectada:
            if a.cantidadInmunes >= 3 and self.nivelAnticuerpos > 70 and random.random() < probabilidadRescate:
                return EstadoCelular.GlobuloRojo
        else:
            if a.cantidadInmunes >=4 and self.nivelAnticuerpos > 80 and random.random() < probabilidadRescate:
                return EstadoCelular.Neutrofilo

        return self.estadoActual

    def aplicarReglasExplosionViral(self, a: 'analisisVecindario') -> EstadoCelular:


        if self.edadEstado >= 2:
            # Regla: Determinar qué tipo de zombie basado en célula original.
            if random.random() < 0.7:
                return EstadoCelular.GlobuloRojoZombie
            else:
                return EstadoCelular.GlobuloBlancoZombie

        return EstadoCelular.ExplosionViral

    def aplicarReglasZombie(self, a: 'analisisVecindario') -> EstadoCelular:
        #Regla: Eliminación masiva debido al sistema inmune.
        if a.cantidadInmunes >= 5 and self.nivelAnticuerpos > 80:
            if self.edadEstado > 20 and random.random() < 0.15:
                return EstadoCelular.TejidoNecrotico

        #Regla: Estado de necrosis natural.
        if self.edadEstado > 50:
            if random.random() < 0.05:
                return EstadoCelular.TejidoNecrotico

        return self.estadoActual

    def aplicarReglasNecrotico(self, a: 'analisisVecindario') -> EstadoCelular:

        if self.edadEstado > 80:
            if random.random() < 0.1:
                return EstadoCelular.Plasma

        return EstadoCelular.TejidoNecrotico

    def aplicarReglasCoagulo(self, a: 'analisisVecindario') -> EstadoCelular:
        #Regla: Se disuelven si no hay amenaza.
        if (self.edadEstado > 25 and a.cantidadZombies == 0 and
            a.cantidadInfectadas == 0):
            if random.random() < 0.2:
                return EstadoCelular.Plasma

        #Regla: Se mantienen mientras haya infección cercana.
        return EstadoCelular.CoaguloSangre

    def establecerSiguienteEstado(self, estado: EstadoCelular):
        self.estadoSiguiente = estado

    def aplicarSiguienteEstado(self):
        if self.estadoActual != self.estadoSiguiente:
            self.estadoActual = self.estadoSiguiente
            self.edadEstado = 0
        else:
            self.edadEstado += 1

        #Regla: Decaimiento natural de parámetros.
        self.cargaViral = max(0, self.cargaViral - 2)
        self.nivelAnticuerpos = max(0, self.nivelAnticuerpos - 1)
        self.memoriaInmune = max(0, self.memoriaInmune - 1)
        self.nivelEnergia = min(100, self.nivelEnergia + 1)

    def obtenerEstadoActual(self) -> EstadoCelular:
        return self.estadoActual

    def establecerEstado(self, estado: EstadoCelular):
        self.estadoActual = estado
        self.estadoSiguiente = estado
        self.edadEstado = 0

    # Getters para parámetros internos
    def obtenerCargaViral(self) -> int:
        return self.cargaViral

    def obtenerNivelAnticuerpos(self) -> int:
        return self.nivelAnticuerpos

    def obtenerMemoriaInmune(self) -> int:
        return self.memoriaInmune

    def obtenerNivelEnergia(self) -> int:
        return self.nivelEnergia


    @dataclass
    class analisisVecindario:
        cantidadVirus: int = 0
        cantidadInfectadas: int = 0
        cantidadZombies: int = 0
        cantidadInmunes: int = 0
        cantidadGlobulosRojos: int = 0
        cantidadPlaquetas: int = 0
        cantidadCoagulos: int = 0
        cantidadPlasma: int = 0
        cargaViralVecindario: int = 0
        totalAnticuerpos: int = 0


### Clase Grilla Torrente Sanguíneo

In [10]:
# @title
import random
import math
# Para correr como archivo .py, descomentar las siguientes dos líneas. Para correr como Collab, comentar las siguientes dos líneas
#from celula_sanguinea import CelulaSanguinea
#from estado_celular import EstadoCelular

class GrillaTorrenteSanguineo:

    #Constantes Fisiológicas.
    _TEMP_NORMAL = 37.0
    _TEMP_MAX_FIEBRE = 42.0
    _UMBRAL_ZOMBIFICACION = 60.0

    def __init__(self, ancho, alto):
        self.__ancho = ancho
        self.__alto = alto
        self.__generacion = 0

        # Grilla bidimensional - inicializada con Plasma por defecto.
        self.__grilla = [[CelulaSanguinea(EstadoCelular.Plasma) for _ in range(ancho)] for _ in range(alto)]

        #Métricas médicas.
        self.__temperaturaCorporal = self._TEMP_NORMAL
        self.__tasaInfeccion = 0.0
        self.__eficienciaInmune = 100.0
        self.__estaZombificado = False
        self.__etapaInfeccion = "Sano"

        #Contadores celulares.
        self.__globulosRojos = 0
        self.__globulosBlancos = 0
        self.__celulasInfectadas = 0
        self.__celulasZombie = 0
        self.__particulasVirus = 0
        self.__celulasInmunesActivas = 0

        self.inicializarTorrenteSanguineo()

    def inicializarTorrenteSanguineo(self):
        self.__generacion = 0
        self.__temperaturaCorporal = self._TEMP_NORMAL

        for fila in range(self.__alto):
            for col in range(self.__ancho):
                aleatorio = random.random()

                # Paredes de vasos sanguíneos (los bordes).
                if fila == 0 or fila == self.__alto - 1 or col == 0 or col == self.__ancho - 1:
                    if random.random() < 0.7:
                        self.__grilla[fila][col] = CelulaSanguinea(EstadoCelular.ParedVaso)
                        continue

                #Distribución normal de células sanguíneas.
                if aleatorio < 0.45:
                    # 45% Glóbulos rojos
                    self.__grilla[fila][col] = CelulaSanguinea(EstadoCelular.GlobuloRojo)
                elif aleatorio < 0.456:
                    # 0.6% Neutrófilos
                    self.__grilla[fila][col] = CelulaSanguinea(EstadoCelular.Neutrofilo)
                elif aleatorio < 0.459:
                    # 0.3% Linfocitos
                    self.__grilla[fila][col] = CelulaSanguinea(EstadoCelular.Linfocito)
                elif aleatorio < 0.460:
                    # 0.1% Macrófagos
                    self.__grilla[fila][col] = CelulaSanguinea(EstadoCelular.Macrofago)
                elif aleatorio < 0.470:
                    # 1% Plaquetas
                    self.__grilla[fila][col] = CelulaSanguinea(EstadoCelular.Plaqueta)
                else:
                    # 53% Plasma
                    self.__grilla[fila][col] = CelulaSanguinea(EstadoCelular.Plasma)

        self.actualizarEstadisticas()

    def introducirVirus(self, fila, col, radio):
        """
            fila: Fila central de la infección
            col: Columna central de la infección
            radio: Radio del área infectada
        """
        for df in range(-radio, radio + 1):
            for dc in range(-radio, radio + 1):
                f = fila + df
                c = col + dc

                if f >= 0 and f < self.__alto and c >= 0 and c < self.__ancho:
                    distancia = math.sqrt(df * df + dc * dc)
                    if distancia <= radio:
                        estado = self.__grilla[f][c].obtenerEstadoActual()

                        #Instrucción de infección células en el área.
                        if estado == EstadoCelular.GlobuloRojo:
                            self.__grilla[f][c].establecerEstado(EstadoCelular.CelulaRojaInfectada)
                        elif estado.esCelulaInmune():
                            self.__grilla[f][c].establecerEstado(EstadoCelular.CelulaBlancaInfectada)
                        elif estado == EstadoCelular.Plasma:
                            self.__grilla[f][c].establecerEstado(EstadoCelular.ParticulaVirus)

    def paso(self):
        #LA ACTUALIZACIÓN DEL TORRENTE SANGUÍNEO ES UN PASO DEL AUTÓMATA.
        #Fase 1: Calcular próximo estado.
        for fila in range(self.__alto):
            for col in range(self.__ancho):
                vecinos = self.obtenerVecinos(fila, col)
                siguienteEstado = self.__grilla[fila][col].calcularSiguienteEstado(vecinos)
                self.__grilla[fila][col].establecerSiguienteEstado(siguienteEstado)

        #Fase 2: Aplicar cambios.
        for fila in range(self.__alto):
            for col in range(self.__ancho):
                self.__grilla[fila][col].aplicarSiguienteEstado()

        self.__generacion += 1
        self.actualizarEstadisticas()
        self.calcularTemperaturaCorporal()
        self.determinarEtapaInfeccion()

    def obtenerVecinos(self, fila, col):
        """
        Se obtienen los 8 vecinos (principio de vecindario de Moore).

        Args:
            fila: Fila de la célula
            col: Columna de la célula

        Returns:
            Lista de 8 vecinos (puede contener None para bordes)
        """
        vecinos = [None] * 8
        indice = 0

        for df in range(-1, 2):
            for dc in range(-1, 2):
                if df == 0 and dc == 0:
                    continue

                f = fila + df
                c = col + dc

                if f >= 0 and f < self.__alto and c >= 0 and c < self.__ancho:
                    vecinos[indice] = self.__grilla[f][c]
                else:
                    vecinos[indice] = None

                indice += 1

        return vecinos

    def actualizarEstadisticas(self):
        self.__globulosRojos = 0
        self.__globulosBlancos = 0
        self.__celulasInfectadas = 0
        self.__celulasZombie = 0
        self.__particulasVirus = 0
        self.__celulasInmunesActivas = 0

        totalCelulas = 0

        for fila in range(self.__alto):
            for col in range(self.__ancho):
                estado = self.__grilla[fila][col].obtenerEstadoActual()

                if estado != EstadoCelular.Plasma and estado != EstadoCelular.ParedVaso:
                    totalCelulas += 1

                if estado == EstadoCelular.GlobuloRojo:
                    self.__globulosRojos += 1
                elif estado.esCelulaInmune():
                    self.__globulosBlancos += 1
                    if (estado == EstadoCelular.NeutrofiloActivado or
                        estado == EstadoCelular.LinfocitoActivado or
                        estado == EstadoCelular.MacrofagoActivado):
                        self.__celulasInmunesActivas += 1
                elif estado.estaInfectada():
                    self.__celulasInfectadas += 1
                elif estado.esZombie():
                    self.__celulasZombie += 1
                elif estado.esVirus():
                    self.__particulasVirus += 1

        #Calcular tasas de infección.
        if totalCelulas > 0:
            self.__tasaInfeccion = ((self.__celulasInfectadas + self.__celulasZombie) / totalCelulas) * 100.0
            if self.__globulosBlancos > 0:
                self.__eficienciaInmune = (self.__celulasInmunesActivas / self.__globulosBlancos) * 100.0
            else:
                self.__eficienciaInmune = 0.0

        #Determinar zombificación.
        self.__estaZombificado = self.__tasaInfeccion >= self._UMBRAL_ZOMBIFICACION

    def calcularTemperaturaCorporal(self):

        #Temperatura base.
        temp = self._TEMP_NORMAL

        #Aumento por infección activa.
        temp += (self.__tasaInfeccion / 100.0) * 4.0

        #Aumento por respuesta inmune (fiebre).
        temp += (self.__celulasInmunesActivas / 100.0) * 1.5

        #Caída si zombificación completa (cuerpo "muerto").
        if self.__estaZombificado:
            temp = self._TEMP_NORMAL - 2.0 + (random.random() * 1.0)


        self.__temperaturaCorporal = max(35.0, min(self._TEMP_MAX_FIEBRE, temp))

    def determinarEtapaInfeccion(self):
        if self.__estaZombificado:
            self.__etapaInfeccion = "ZOMBIFICADO"
        elif self.__tasaInfeccion > 40:
            self.__etapaInfeccion = "CRÍTICO"
        elif self.__tasaInfeccion > 20:
            self.__etapaInfeccion = "SEVERO"
        elif self.__tasaInfeccion > 5:
            self.__etapaInfeccion = "MODERADO"
        elif self.__celulasInfectadas > 0 or self.__particulasVirus > 0:
            self.__etapaInfeccion = "TEMPRANO"
        else:
            self.__etapaInfeccion = "SALUDABLE"


    def obtenerEstadoCelula(self, fila, col):
        if fila >= 0 and fila < self.__alto and col >= 0 and col < self.__ancho:
            return self.__grilla[fila][col].obtenerEstadoActual()
        return EstadoCelular.Plasma

    def obtenerAncho(self):
        return self.__ancho

    def obtenerAlto(self):
        return self.__alto

    def obtenerGeneracion(self):
        return self.__generacion

    def obtenerTemperaturaCorporal(self):
        return self.__temperaturaCorporal

    def obtenerTasaInfeccion(self):
        return self.__tasaInfeccion

    def obtenerEficienciaInmune(self):
        return self.__eficienciaInmune

    def obtenerEstaZombificado(self):
        return self.__estaZombificado

    def obtenerEtapaInfeccion(self):
        return self.__etapaInfeccion

    def obtenerGlobulosRojos(self):
        return self.__globulosRojos

    def obtenerGlobulosBlancos(self):
        return self.__globulosBlancos

    def obtenerCelulasInfectadas(self):
        return self.__celulasInfectadas

    def obtenerCelulasZombie(self):
        return self.__celulasZombie

    def obtenerParticulasVirus(self):
        return self.__particulasVirus

    def obtenerCelulasInmunesActivas(self):
        return self.__celulasInmunesActivas

### Clase Simulador

La siguiente celda de código corresponde a la interfaz diseñada para ejecutarse en un entorno de PC directo, por lo que no se ejeccuta en este Colab.

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import threading
import time
import random
import math

from grilla_torrente_sanguineo import GrillaTorrenteSanguineo
from estado_celular import EstadoCelular


class Simulador:
    """
    SIMULADOR DE INFECCIÓN ZOMBIE A NIVEL SANGUÍNEO

    Visualiza el torrente sanguíneo humano durante una infección zombie
    con dashboard médico en tiempo real.
    """

    ANCHO_SANGRE = 120
    ALTO_SANGRE = 80
    TAMANO_CELULA = 7

    def __init__(self):
        self.root = tk.Tk()
        self.root.title("🩸 Simulador de Infección Zombie")
        self.root.configure(bg="#f8f9fa")

        #Dimensiones ventana
        self.root.geometry("1350x750")

        self.torrente_sanguineo = GrillaTorrenteSanguineo(self.ANCHO_SANGRE, self.ALTO_SANGRE)
        self.ejecutando = False
        self.hilo_simulacion = None

        #Crear contenedor scrollable y dentro él la interfaz.
        self._crear_contenedor_scrollable()

        #Construcción de la interfaz dentro del contenedor scrollable.
        self.configurar_interfaz()

        self.root.protocol("WM_DELETE_WINDOW", self.cerrar)

    def _crear_contenedor_scrollable(self):
        #Interfaz principal que tendrá el contenido y tiene scrollbar.
        self.main_canvas = tk.Canvas(self.root, bg="#f8f9fa", highlightthickness=0)
        self.main_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        #Scrollbar vertical.
        self.v_scroll = ttk.Scrollbar(self.root, orient=tk.VERTICAL, command=self.main_canvas.yview)
        self.v_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        self.main_canvas.configure(yscrollcommand=self.v_scroll.set)

        #Frame que continue la UI.
        self.contenedor_principal = tk.Frame(self.main_canvas, bg="#f8f9fa")
        self.canvas_window = self.main_canvas.create_window((0, 0), window=self.contenedor_principal, anchor="nw")

        #Cuando el tamaño del frame cambie, actualizar scrollregion.
        def _on_frame_configure(event):
            self.main_canvas.configure(scrollregion=self.main_canvas.bbox("all"))

        self.contenedor_principal.bind("<Configure>", _on_frame_configure)

        def _on_canvas_resize(event):
            canvas_width = event.width
            self.main_canvas.itemconfigure(self.canvas_window, width=canvas_width)

        self.main_canvas.bind("<Configure>", _on_canvas_resize)

        def _on_mousewheel(event):
            if hasattr(event, 'delta') and event.delta:
                self.main_canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")
            else:
                if event.num == 4:
                    self.main_canvas.yview_scroll(-3, "units")
                elif event.num == 5:
                    self.main_canvas.yview_scroll(3, "units")

        self.root.bind_all("<MouseWheel>", _on_mousewheel)
        self.root.bind_all("<Button-4>", _on_mousewheel)
        self.root.bind_all("<Button-5>", _on_mousewheel)

    def configurar_interfaz(self):

        #Configuración de toda la interface dentro del contenedor principal.
        parent = self.contenedor_principal

        self.crear_encabezado(parent)
        # Panel de controles
        self.crear_panel_controles(parent)

        #Contenedor horizontal para simulación + dashboard.
        contenedor_horizontal = tk.Frame(parent, bg="#f8f9fa")
        contenedor_horizontal.pack(fill=tk.BOTH, expand=True, pady=8, padx=10)

        #Panel de simulación (a la izquierda).
        self.crear_panel_simulacion(contenedor_horizontal)

        #Dashboard médico (a la derecha).
        self.crear_dashboard_medico(contenedor_horizontal)

        self.actualizar_visualizacion()

    def crear_encabezado(self, parent):
        header = tk.Frame(parent, bg="#ffffff", relief=tk.FLAT, bd=0)
        header.pack(fill=tk.X, pady=(0, 10))

        linea_top = tk.Frame(header, bg="#e74c3c", height=3)
        linea_top.pack(fill=tk.X)

        #Contenido del header.
        contenido = tk.Frame(header, bg="#ffffff")
        contenido.pack(fill=tk.X, pady=12, padx=25)

        #Título.
        titulo = tk.Label(
            contenido,
            text="🧬 Simulador de Infección Zombie",
            font=("Helvetica", 20, "bold"),
            bg="#ffffff",
            fg="#2c3e50"
        )
        titulo.pack(side=tk.LEFT)

        #Subtítulo.
        subtitulo = tk.Label(
            contenido,
            text="Visualización de Autómata Celular en Torrente Sanguíneo",
            font=("Helvetica", 10),
            bg="#ffffff",
            fg="#7f8c8d"
        )
        subtitulo.pack(side=tk.LEFT, padx=15)

    def crear_panel_controles(self, parent):

        panel = tk.Frame(parent, bg="#ffffff", relief=tk.FLAT)
        panel.pack(fill=tk.X, pady=(0, 10))

        sombra = tk.Frame(panel, bg="#e0e0e0", height=1)
        sombra.pack(fill=tk.X, side=tk.BOTTOM)

        contenido = tk.Frame(panel, bg="#ffffff")
        contenido.pack(fill=tk.X, pady=10, padx=20)

        #Botones.
        botones = [
            ("▶ Iniciar", "#27ae60", self.iniciar),
            ("⏸ Pausar", "#f39c12", self.pausar),
            ("⏭ Paso", "#95a5a6", self.paso),
            ("🔄 Reiniciar", "#3498db", self.reiniciar),
            ("💉 Inyectar Virus", "#e74c3c", self.inyectar_virus),
        ]

        for texto, color, comando in botones:
            btn = tk.Button(
                contenido,
                text=texto,
                font=("Helvetica", 10, "bold"),
                bg=color,
                fg="white",
                activebackground=self.oscurecer_color(color),
                activeforeground="white",
                relief=tk.FLAT,
                padx=20,
                pady=8,
                cursor="hand2",
                command=comando
            )
            btn.pack(side=tk.LEFT, padx=4)

            btn.bind("<Enter>", lambda e, b=btn, c=color: b.config(bg=self.oscurecer_color(c)))
            btn.bind("<Leave>", lambda e, b=btn, c=color: b.config(bg=c))

    #Intento de efecto hover.
    def oscurecer_color(self, hex_color):
        hex_color = hex_color.lstrip('#')
        r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        r, g, b = max(0, r-30), max(0, g-30), max(0, b-30)
        return f'#{r:02x}{g:02x}{b:02x}'

    def crear_panel_simulacion(self, parent):

        panel = tk.Frame(parent, bg="#ffffff", relief=tk.FLAT)
        panel.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0, 8))

        #Panel.
        titulo = tk.Label(
            panel,
            text="🔬 Vista del Torrente Sanguíneo",
            font=("Helvetica", 13, "bold"),
            bg="#ffffff",
            fg="#2c3e50"
        )
        titulo.pack(pady=(10, 8), padx=15, anchor="w")

        canvas_container = tk.Frame(panel, bg="#ecf0f1", relief=tk.FLAT, bd=1)
        canvas_container.pack(padx=15, pady=(0, 10))

        self.canvas = tk.Canvas(
            canvas_container,
            width=self.ANCHO_SANGRE * self.TAMANO_CELULA,
            height=self.ALTO_SANGRE * self.TAMANO_CELULA,
            bg="#1a1a1a",
            highlightthickness=0
        )
        self.canvas.pack(padx=2, pady=2)

        #Leyenda.
        leyenda_frame = tk.Frame(panel, bg="#ffffff")
        leyenda_frame.pack(pady=(0, 10), padx=15, fill=tk.X)

        leyendas = [
            ("●", "#dc143c", "Eritrocitos"),
            ("●", "#6495ed", "Leucocitos"),
            ("●", "#ffeb3b", "Virus"),
            ("●", "#ff8c00", "Infectadas"),
            ("●", "#8b0000", "Zombie"),
            ("●", "#ffe0e0", "Plasma"),
        ]

        for simbolo, color, texto in leyendas:
            item_frame = tk.Frame(leyenda_frame, bg="#ffffff")
            item_frame.pack(side=tk.LEFT, padx=8)

            circulo = tk.Label(
                item_frame,
                text=simbolo,
                font=("Helvetica", 14),
                fg=color,
                bg="#ffffff"
            )
            circulo.pack(side=tk.LEFT, padx=(0, 4))

            label = tk.Label(
                item_frame,
                text=texto,
                font=("Helvetica", 9),
                fg="#34495e",
                bg="#ffffff"
            )
            label.pack(side=tk.LEFT)

    def crear_dashboard_medico(self, parent):
        """Crea el dashboard médico."""
        dashboard = tk.Frame(parent, bg="#ffffff", relief=tk.FLAT, width=330)
        dashboard.pack(side=tk.RIGHT, fill=tk.BOTH, padx=(8, 0))
        dashboard.pack_propagate(False)

        #Título.
        titulo = tk.Label(
            dashboard,
            text="📊 Monitoreo Vital",
            font=("Helvetica", 14, "bold"),
            bg="#ffffff",
            fg="#2c3e50"
        )
        titulo.pack(pady=(12, 10), padx=15, anchor="w")

        #Barra de temperatura.
        self.crear_metrica_temperatura(dashboard)

        #Barra nivel
        self.crear_metrica_infeccion(dashboard)

        #Barra de Zombificación.
        self.crear_metrica_zombificacion(dashboard)

        #Sistema Inmune.
        self.crear_metrica_inmune(dashboard)

        #Conteo Celular.
        self.crear_metrica_conteo(dashboard)

    def crear_metrica_temperatura(self, parent):
        panel = tk.Frame(parent, bg="#ffffff", relief=tk.FLAT)
        panel.pack(fill=tk.X, padx=15, pady=5)

        tk.Frame(panel, bg="#ecf0f1", height=1).pack(fill=tk.X, pady=(0, 6))

        label = tk.Label(
            panel,
            text="🌡️ Temperatura Corporal",
            font=("Helvetica", 10, "bold"),
            bg="#ffffff",
            fg="#34495e"
        )
        label.pack(anchor="w")

        self.etiqueta_temp = tk.Label(
            panel,
            text="37.0°C",
            font=("Helvetica", 24, "bold"),
            bg="#ffffff",
            fg="#27ae60"
        )
        self.etiqueta_temp.pack(anchor="w", pady=(3, 3))

        #Barra de progreso personalizada.
        barra_frame = tk.Frame(panel, bg="#ecf0f1", height=6)
        barra_frame.pack(fill=tk.X, pady=4)

        self.barra_temp_fill = tk.Frame(barra_frame, bg="#27ae60", height=6)
        self.barra_temp_fill.place(x=0, y=0, relwidth=0.88, relheight=1)

        rango = tk.Label(
            panel,
            text="Normal: 36-38°C",
            font=("Helvetica", 8),
            bg="#ffffff",
            fg="#95a5a6"
        )
        rango.pack(anchor="w", pady=(2, 0))

    def crear_metrica_infeccion(self, parent):
        panel = tk.Frame(parent, bg="#ffffff", relief=tk.FLAT)
        panel.pack(fill=tk.X, padx=15, pady=5)

        tk.Frame(panel, bg="#ecf0f1", height=1).pack(fill=tk.X, pady=(0, 6))

        label = tk.Label(
            panel,
            text="🦠 Nivel de Infección",
            font=("Helvetica", 10, "bold"),
            bg="#ffffff",
            fg="#34495e"
        )
        label.pack(anchor="w")

        self.etiqueta_infeccion = tk.Label(
            panel,
            text="0.0%",
            font=("Helvetica", 24, "bold"),
            bg="#ffffff",
            fg="#95a5a6"
        )
        self.etiqueta_infeccion.pack(anchor="w", pady=(3, 3))

        barra_frame = tk.Frame(panel, bg="#ecf0f1", height=6)
        barra_frame.pack(fill=tk.X, pady=4)

        self.barra_infeccion_fill = tk.Frame(barra_frame, bg="#95a5a6", height=6)
        self.barra_infeccion_fill.place(x=0, y=0, relwidth=0, relheight=1)

    def crear_metrica_zombificacion(self, parent):
        panel = tk.Frame(parent, bg="#ffffff", relief=tk.FLAT)
        panel.pack(fill=tk.X, padx=15, pady=5)

        tk.Frame(panel, bg="#ecf0f1", height=1).pack(fill=tk.X, pady=(0, 6))

        label = tk.Label(
            panel,
            text="🧟 Estado de Zombificación",
            font=("Helvetica", 10, "bold"),
            bg="#ffffff",
            fg="#34495e"
        )
        label.pack(anchor="w")

        self.etiqueta_estado_zombie = tk.Label(
            panel,
            text="✓ Saludable",
            font=("Helvetica", 14, "bold"),
            bg="#ffffff",
            fg="#27ae60"
        )
        self.etiqueta_estado_zombie.pack(anchor="w", pady=(3, 2))

        self.etiqueta_etapa = tk.Label(
            panel,
            text="Etapa: Normal",
            font=("Helvetica", 9),
            bg="#ffffff",
            fg="#95a5a6"
        )
        self.etiqueta_etapa.pack(anchor="w")

    def crear_metrica_inmune(self, parent):
        """Crea la métrica del sistema inmune."""
        panel = tk.Frame(parent, bg="#ffffff", relief=tk.FLAT)
        panel.pack(fill=tk.X, padx=15, pady=5)

        tk.Frame(panel, bg="#ecf0f1", height=1).pack(fill=tk.X, pady=(0, 6))

        label = tk.Label(
            panel,
            text="🛡️ Sistema Inmune",
            font=("Helvetica", 10, "bold"),
            bg="#ffffff",
            fg="#34495e"
        )
        label.pack(anchor="w")

        barra_frame = tk.Frame(panel, bg="#ecf0f1", height=6)
        barra_frame.pack(fill=tk.X, pady=6)

        self.barra_inmune_fill = tk.Frame(barra_frame, bg="#3498db", height=6)
        self.barra_inmune_fill.place(x=0, y=0, relwidth=0, relheight=1)

        self.etiqueta_inmune = tk.Label(
            panel,
            text="Eficiencia: 0%",
            font=("Helvetica", 9),
            bg="#ffffff",
            fg="#3498db"
        )
        self.etiqueta_inmune.pack(anchor="w")

    def crear_metrica_conteo(self, parent):
        """Crea la métrica de conteo celular."""
        panel = tk.Frame(parent, bg="#ffffff", relief=tk.FLAT)
        panel.pack(fill=tk.X, padx=15, pady=5)

        tk.Frame(panel, bg="#ecf0f1", height=1).pack(fill=tk.X, pady=(0, 6))

        label = tk.Label(
            panel,
            text="🔬 Análisis Celular",
            font=("Helvetica", 10, "bold"),
            bg="#ffffff",
            fg="#34495e"
        )
        label.pack(anchor="w", pady=(0, 6))

        #Contadores para sección.
        self.etiqueta_globulos_rojos = self.crear_contador(panel, "●", "#dc143c", "Eritrocitos: 0")
        self.etiqueta_globulos_blancos = self.crear_contador(panel, "●", "#6495ed", "Leucocitos: 0")
        self.etiqueta_celulas_infectadas = self.crear_contador(panel, "●", "#ff8c00", "Infectadas: 0")
        self.etiqueta_virus = self.crear_contador(panel, "●", "#ffeb3b", "Virus: 0")
        self.etiqueta_zombie = self.crear_contador(panel, "●", "#8b0000", "Zombie: 0")

    def crear_contador(self, parent, simbolo, color, texto_inicial):
        frame = tk.Frame(parent, bg="#ffffff")
        frame.pack(fill=tk.X, pady=2)

        circulo = tk.Label(
            frame,
            text=simbolo,
            font=("Helvetica", 10),
            fg=color,
            bg="#ffffff"
        )
        circulo.pack(side=tk.LEFT, padx=(0, 6))

        label = tk.Label(
            frame,
            text=texto_inicial,
            font=("Helvetica", 9),
            bg="#ffffff",
            fg="#34495e",
            anchor="w"
        )
        label.pack(side=tk.LEFT, fill=tk.X)

        return label

    def rgb_a_hex(self, rgb_tuple):
        return f'#{rgb_tuple[0]:02x}{rgb_tuple[1]:02x}{rgb_tuple[2]:02x}'

    def dibujar_sangre(self):
        self.canvas.delete("all")

        for fila in range(self.torrente_sanguineo.obtenerAlto()):
            for col in range(self.torrente_sanguineo.obtenerAncho()):
                estado = self.torrente_sanguineo.obtenerEstadoCelula(fila, col)
                color = self.rgb_a_hex(estado.obtenerColor())

                x1 = col * self.TAMANO_CELULA
                y1 = fila * self.TAMANO_CELULA
                x2 = x1 + self.TAMANO_CELULA - 1
                y2 = y1 + self.TAMANO_CELULA - 1

                self.canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="")

    def actualizar_visualizacion(self):
        self.dibujar_sangre()

        #Actualización Temperatura.
        temp = self.torrente_sanguineo.obtenerTemperaturaCorporal()
        self.etiqueta_temp.config(text=f"{temp:.1f}°C")

        #Actualizar barra y color de temperatura.
        temp_percent = min(1.0, max(0.0, (temp - 36) / 6))
        self.barra_temp_fill.place(relwidth=temp_percent)

        if temp > 39.5:
            color_temp = "#e74c3c"
        elif temp > 38.0:
            color_temp = "#f39c12"
        elif temp < 36.0:
            color_temp = "#3498db"
        else:
            color_temp = "#27ae60"

        self.etiqueta_temp.config(fg=color_temp)
        self.barra_temp_fill.config(bg=color_temp)

        #Infección.
        tasa_infeccion = self.torrente_sanguineo.obtenerTasaInfeccion()
        self.etiqueta_infeccion.config(text=f"{tasa_infeccion:.1f}%")
        self.barra_infeccion_fill.place(relwidth=min(1.0, max(0.0, tasa_infeccion / 100.0)))

        if tasa_infeccion > 70:
            color_inf = "#c0392b"
        elif tasa_infeccion > 40:
            color_inf = "#e74c3c"
        elif tasa_infeccion > 10:
            color_inf = "#f39c12"
        else:
            color_inf = "#95a5a6"

        self.etiqueta_infeccion.config(fg=color_inf)
        self.barra_infeccion_fill.config(bg=color_inf)

        #Actualización Zombificación.
        if self.torrente_sanguineo.obtenerEstaZombificado():
            self.etiqueta_estado_zombie.config(text="☠ Zombificado", fg="#c0392b")
        else:
            self.etiqueta_estado_zombie.config(text="✓ Saludable", fg="#27ae60")

        self.etiqueta_etapa.config(text=f"Etapa: {self.torrente_sanguineo.obtenerEtapaInfeccion()}")

        #Actualización Sistema Inmune.
        eficiencia = self.torrente_sanguineo.obtenerEficienciaInmune()
        self.barra_inmune_fill.place(relwidth=min(1.0, max(0.0, eficiencia / 100.0)))
        self.etiqueta_inmune.config(text=f"Eficiencia: {eficiencia:.0f}%")

        #Actualización del conteo celular.
        self.etiqueta_globulos_rojos.config(
            text=f"Eritrocitos: {self.torrente_sanguineo.obtenerGlobulosRojos()}"
        )
        self.etiqueta_globulos_blancos.config(
            text=f"Leucocitos: {self.torrente_sanguineo.obtenerGlobulosBlancos()} "
                 f"(Activos: {self.torrente_sanguineo.obtenerCelulasInmunesActivas()})"
        )
        self.etiqueta_celulas_infectadas.config(
            text=f"Infectadas: {self.torrente_sanguineo.obtenerCelulasInfectadas()}"
        )
        self.etiqueta_virus.config(
            text=f"Virus: {self.torrente_sanguineo.obtenerParticulasVirus()}"
        )
        self.etiqueta_zombie.config(
            text=f"Zombie: {self.torrente_sanguineo.obtenerCelulasZombie()}"
        )

    def loop_simulacion(self):
        while self.ejecutando:
            self.torrente_sanguineo.paso()
            #actualizar GUI desde thread con after
            self.root.after(0, self.actualizar_visualizacion)
            self.root.after(0, self.verificar_zombificacion)
            time.sleep(0.2)

    #Iniciación de la simmulación.
    def iniciar(self):
        if not self.ejecutando:
            self.ejecutando = True
            self.hilo_simulacion = threading.Thread(target=self.loop_simulacion, daemon=True)
            self.hilo_simulacion.start()

    def pausar(self):
        self.ejecutando = False

    def paso(self):
        self.torrente_sanguineo.paso()
        self.actualizar_visualizacion()
        self.verificar_zombificacion()

    def reiniciar(self):
        self.ejecutando = False
        time.sleep(0.3)
        self.torrente_sanguineo = GrillaTorrenteSanguineo(self.ANCHO_SANGRE, self.ALTO_SANGRE)
        self.actualizar_visualizacion()

    def inyectar_virus(self):
        radio = 3 + int(random.random() * 5)
        margen = radio + 5
        fila = margen + int(random.random() * (self.ALTO_SANGRE - 2 * margen))
        col = margen + int(random.random() * (self.ANCHO_SANGRE - 2 * margen))

        self.torrente_sanguineo.introducirVirus(fila, col, radio)

        messagebox.showinfo(
            "Virus Inyectado",
            f"💉 Virus zombie introducido en el sistema\n\n"
            f"Posición: Fila {fila}, Columna {col}\n"
            f"Radio de infección: {radio} células\n\n"
            f"Observe la propagación de la infección."
        )

        self.actualizar_visualizacion()

    def verificar_zombificacion(self):
        if self.torrente_sanguineo.obtenerEstaZombificado() and self.ejecutando:
            self.ejecutando = False

            messagebox.showerror(
                "Zombificación Completa",
                f"☠ CONVERSIÓN A ZOMBIE COMPLETA ☠\n\n"
                f"Infección: {self.torrente_sanguineo.obtenerTasaInfeccion():.1f}%\n"
                f"Temperatura: {self.torrente_sanguineo.obtenerTemperaturaCorporal():.1f}°C\n"
                f"Generación: {self.torrente_sanguineo.obtenerGeneracion()}\n\n"
                f"El sujeto ha sido completamente zombificado."
            )

    def cerrar(self):
        self.ejecutando = False
        time.sleep(0.3)
        self.root.destroy()

    def ejecutar(self):
        self.root.mainloop()


if __name__ == "__main__":
    simulador = Simulador()
    simulador.ejecutar()

Ahora bien, se propone el BORRADOR de una alternativa para esta interfaz en el entorno de colab.

In [17]:
# Esta es la interfaz gráfica diseñada para correr en Google Collab - Propuesta por el estudiante Juan Piña / Es importante recalccar que es el INTENTO de esa versión.
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import threading
import time
import random

class SimuladorColab:
    ANCHO_SANGRE = 120
    ALTO_SANGRE = 80

    def __init__(self):
        self.torrente = GrillaTorrenteSanguineo(self.ANCHO_SANGRE, self.ALTO_SANGRE)
        self.ejecutando = False
        self.hilo = None

        self.estado_a_rgb = {}
        for estado in EstadoCelular:
            try:
                rgb = estado.obtenerColor()  #  (r,g,b) 0-255 o 0-1
            except Exception:
                # si el Enum devuelve algo distinto
                rgb = (255, 255, 255)
            # detectar si ya está normalizado
            if all(0 <= x <= 1 for x in rgb):
                rgb_norm = tuple(float(x) for x in rgb)
            else:
                # asumimos 0-255
                rgb_norm = (rgb[0]/255.0, rgb[1]/255.0, rgb[2]/255.0)
            self.estado_a_rgb[estado] = rgb_norm

        # UI
        self.crear_interfaz()

    def crear_interfaz(self):
        #Output para la figura
        self.out_plot = widgets.Output(layout={'border': '1px solid #ccc'})

        #Widgets del dashboard
        self.lbl_temp = widgets.HTML("<h2>37.0 °C</h2>")
        self.bar_temp = widgets.FloatProgress(value=37.0, min=35.0, max=42.0)
        box_temp = widgets.VBox([widgets.HTML("<b>🌡️ Temperatura Corporal</b>"), self.lbl_temp, self.bar_temp])
        box_temp.layout = widgets.Layout(border='1px solid #ddd', padding='5px', margin='5px')

        self.lbl_inf = widgets.HTML("<h2>0.0%</h2>")
        self.bar_inf = widgets.FloatProgress(value=0.0, min=0.0, max=100.0)
        box_inf = widgets.VBox([widgets.HTML("<b>🦠 Nivel de Infección</b>"), self.lbl_inf, self.bar_inf])
        box_inf.layout = widgets.Layout(border='1px solid #ddd', padding='5px', margin='5px')

        self.lbl_zombie = widgets.HTML("<h3 style='color:green'>✓ Saludable</h3>")
        self.lbl_etapa = widgets.HTML("Etapa: Saludable")
        box_zombie = widgets.VBox([widgets.HTML("<b>🧟 Estado Zombie</b>"), self.lbl_zombie, self.lbl_etapa])
        box_zombie.layout = widgets.Layout(border='1px solid #ddd', padding='5px', margin='5px')

        self.lbl_inmune = widgets.HTML("Eficiencia: 100%")
        self.bar_inmune = widgets.FloatProgress(value=100.0, min=0.0, max=100.0)
        box_inmune = widgets.VBox([widgets.HTML("<b>🛡️ Sistema Inmune</b>"), self.lbl_inmune, self.bar_inmune])
        box_inmune.layout = widgets.Layout(border='1px solid #ddd', padding='5px', margin='5px')

        self.html_conteo = widgets.HTML("Cargando...")
        box_conteo = widgets.VBox([widgets.HTML("<b>🔬 Análisis Celular</b>"), self.html_conteo])
        box_conteo.layout = widgets.Layout(border='1px solid #ddd', padding='5px', margin='5px')

        panel_dashboard = widgets.VBox([
            widgets.HTML("<h3>📊 Monitoreo Vital</h3><hr>"),
            box_temp, box_inf, box_zombie, box_inmune, box_conteo
        ])
        panel_dashboard.layout = widgets.Layout(width='320px')

        #Controles
        self.btn_iniciar = widgets.Button(description="▶ Iniciar", button_style='success')
        self.btn_pausar = widgets.Button(description="⏸ Pausar", button_style='warning')
        self.btn_paso = widgets.Button(description="⏭ Paso")
        self.btn_virus = widgets.Button(description="💉 Inyectar Virus", button_style='danger')
        self.btn_reiniciar = widgets.Button(description="🔄 Reiniciar", button_style='primary')

        self.btn_iniciar.on_click(self.accion_iniciar)
        self.btn_pausar.on_click(self.accion_pausar)
        self.btn_paso.on_click(self.accion_paso)
        self.btn_virus.on_click(self.accion_inyectar)
        self.btn_reiniciar.on_click(self.accion_reiniciar)

        panel_controles = widgets.HBox([self.btn_iniciar, self.btn_pausar, self.btn_paso, self.btn_virus, self.btn_reiniciar])

        contenedor_principal = widgets.HBox([self.out_plot, panel_dashboard], layout=widgets.Layout(align_items='flex-start'))
        self.ui_completa = widgets.VBox([
            widgets.HTML("<h1>🧬 Simulador de Infección Zombie (Colab Edition)</h1>"),
            panel_controles,
            contenedor_principal
        ])

        display(self.ui_completa)
        #dibujar por primera vez
        self.actualizar_vista()

    def obtener_imagen_rgb(self):
        h = self.torrente.obtenerAlto()
        w = self.torrente.obtenerAncho()
        img = np.zeros((h, w, 3), dtype=np.float32)

        for f in range(h):
            for c in range(w):
                estado = self.torrente.obtenerEstadoCelula(f, c)
                # estado puede ser Enum o un objeto; lo maneja defensivamente
                if hasattr(estado, 'obtenerColor'):
                    rgb = estado.obtenerColor()
                elif hasattr(estado, 'value') and hasattr(estado.value, 'obtenerColor'):
                    rgb = estado.value.obtenerColor()
                else:
                    rgb = (255, 255, 255)

                if all(0 <= x <= 1 for x in rgb):
                    rgb_norm = tuple(float(x) for x in rgb)
                else:
                    rgb_norm = (rgb[0]/255.0, rgb[1]/255.0, rgb[2]/255.0)

                img[f, c, :] = rgb_norm

        return img

    def actualizar_vista(self):
        #Re-dibuja la figura entera dentro del Output (compatible con Colab)
        img_rgb = self.obtener_imagen_rgb()

        with self.out_plot:
            clear_output(wait=True)
            fig, ax = plt.subplots(figsize=(9, 6))
            ax.imshow(img_rgb, interpolation='nearest', aspect='auto')
            ax.set_title("🔬 Vista del Torrente Sanguíneo", fontsize=14)
            ax.axis('off')
            plt.tight_layout()
            display(fig)
            plt.close(fig)

        #actualizar dashboard
        temp = self.torrente.obtenerTemperaturaCorporal()
        self.lbl_temp.value = f"<h2>{temp:.1f} °C</h2>"
        self.bar_temp.value = temp

        tasa = self.torrente.obtenerTasaInfeccion()
        self.lbl_inf.value = f"<h2>{tasa:.1f}%</h2>"
        self.bar_inf.value = tasa

        is_z = self.torrente.obtenerEstaZombificado()
        self.lbl_zombie.value = "<h3 style='color:red'>☠ ZOMBIFICADO</h3>" if is_z else "<h3 style='color:green'>✓ Saludable</h3>"
        self.lbl_etapa.value = f"Etapa: {self.torrente.obtenerEtapaInfeccion()}"

        eficiencia = self.torrente.obtenerEficienciaInmune()
        self.lbl_inmune.value = f"Eficiencia: {eficiencia:.1f}%"
        self.bar_inmune.value = eficiencia

        stats_html = f"""
        <ul style="list-style-type: none; padding: 0;">
            <li><span style="color:#dc143c">●</span> Eritrocitos: {self.torrente.obtenerGlobulosRojos()}</li>
            <li><span style="color:#6495ed">●</span> Leucocitos: {self.torrente.obtenerGlobulosBlancos()}</li>
            <li><span style="color:#ff8c00">●</span> Infectadas: {self.torrente.obtenerCelulasInfectadas()}</li>
            <li><span style="color:#ffeb3b">●</span> Virus: {self.torrente.obtenerParticulasVirus()}</li>
            <li><span style="color:#8b0000">●</span> <b>Zombies: {self.torrente.obtenerCelulasZombie()}</b></li>
        </ul>
        """
        self.html_conteo.value = stats_html

    #Loop y controles
    def loop_simulacion(self):
        while self.ejecutando:
            self.torrente.paso()
            self.actualizar_vista()
            if self.torrente.obtenerEstaZombificado():
                self.ejecutando = False
                with self.out_plot:
                    print("⚠️ ALERTA: ZOMBIFICACIÓN COMPLETA ⚠️")
            time.sleep(0.12)

    def accion_iniciar(self, b):
        if not self.ejecutando:
            self.ejecutando = True
            self.hilo = threading.Thread(target=self.loop_simulacion, daemon=True)
            self.hilo.start()

    def accion_pausar(self, b):
        self.ejecutando = False

    def accion_paso(self, b):
        self.ejecutando = False
        self.torrente.paso()
        self.actualizar_vista()

    def accion_inyectar(self, b):
        r = random.randint(3, 7)
        fil = random.randint(10, self.ALTO_SANGRE - 10)
        col = random.randint(10, self.ANCHO_SANGRE - 10)
        self.torrente.introducirVirus(fil, col, r)
        self.actualizar_vista()

    def accion_reiniciar(self, b):
        self.ejecutando = False
        time.sleep(0.2)
        self.torrente = GrillaTorrenteSanguineo(self.ANCHO_SANGRE, self.ALTO_SANGRE)
        self.actualizar_vista()

# Crear e iniciar el app
app = SimuladorColab()
